In [1]:
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import pandas as pd
from enum import Enum
import graphviz as gv
from langchain.prompts.chat import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
import json
from IPython.display import JSON
from tqdm.notebook import tqdm

load_dotenv()

True

In [2]:
llm = ChatOpenAI(model_name="gpt-4")

# A List of Haunted Houses
Let's try a more "top-down" approach, where we first generate a dataset of haunted houses that we will later use to populate with ghosts
individually.

In [3]:
class HauntedHouse(BaseModel):
    name: str
    address: str
    type: str
    description: str
    number_of_ghosts: int
    year_built: int
    year_abandoned: int


class HauntedHouseListings(BaseModel):
    location: str
    haunted_houses: list[HauntedHouse]


pydantic_parser = PydanticOutputParser(pydantic_object=HauntedHouseListings)

PROMPT_FORMAT = """You are a creative AI agent with a vivid imagination and a love of the macabre.
Create a list of {house_count} haunted houses in {location}.
The number of ghosts that reside in each house should follow a Poisson distribution with mean {mean_ghost_count}.
{format_instructions}
The output should not deviate from this format in any way and every house should be listed.
No comments or ellipsis should appear in the output.
"""

prompt = PromptTemplate(
    template=PROMPT_FORMAT,
    input_variables=["house_count", "location", "mean_ghost_count"],
    partial_variables={
        "format_instructions": pydantic_parser.get_format_instructions()
    },
)

chain = LLMChain(llm=llm, prompt=prompt, output_parser=pydantic_parser, verbose=True)

haunted_houses = chain.run(
    location="Covington Georgia", house_count=30, mean_ghost_count=2.5
)



> Entering new LLMChain chain...
Prompt after formatting:
You are a creative AI agent with a vivid imagination and a love of the macabre.
Create a list of 30 haunted houses in Covington Georgia.
The number of ghosts that reside in each house should follow a Poisson distribution with mean 2.5.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"HauntedHouse": {"properties": {"name": {"title": "Name", "type": "string"}, "address": {"title": "Address", "type": "string"}, "type": {"title": "Type", "type": "string"}, "description": {"title": "Description", "type": "string"}, "number_of

In [5]:
haunted_houses_df = pd.DataFrame(
    map(BaseModel.model_dump, haunted_houses.haunted_houses)
)
haunted_houses_df

,name,address,type,description,number_of_ghosts,year_built,year_abandoned
0,The Shadowy Estate,123 Elm Street,Victorian Mansion,A hauntingly beautiful Victorian mansion with ...,3,1875,1940
1,The Whispering Farmhouse,456 Main Road,Farmhouse,An eerie farmhouse where whispers echo through...,2,1890,1960
2,The Shrieking Manor,789 Oak Lane,Manor,A grand manor where bone-chilling shrieks pier...,1,1860,1910
3,The Phantom Homestead,321 Pine Drive,Colonial House,A colonial homestead haunted by phantoms of it...,3,1790,1930
4,The Dreadful Bungalow,654 Willow Way,Bungalow,A quaint bungalow that holds dreadful secrets.,1,1920,1970
5,The Spectral Cottage,987 Maple Avenue,Cottage,A charming cottage that is home to spectral ap...,4,1840,1900
6,The Haunting Hacienda,246 Birch Boulevard,Hacienda,A rustic hacienda that harbors chilling haunti...,2,1800,1950
7,The Ghastly Grange,864 Cedar Court,Grange,A historic grange filled with ghastly appariti...,5,1850,1920
8,The Spooky Shack,753 Aspen Alley,Shack,A dilapidated shack that sends shivers down yo...,1,1910,1955
9,The Creepy Cabin,159 Hickory Heights,Cabin,A secluded cabin with a creepy ambience.,3,1930,2000


In [6]:
haunted_houses_df.number_of_ghosts.mean()

2.5517241379310347

## Generating Ghosts

### Reintroducing our Scariness Scale

In [10]:
SCARINESS_SCALE = [
    {
        "level": 1,
        "name": "Benevolent Specter",
        "description": "Exhibits non-threatening, socially positive behavior patterns.",
    },
    {
        "level": 2,
        "name": "Auditory Manifestation",
        "description": "Engages primarily in acoustic disturbances, evoking mild unease.",
    },
    {
        "level": 3,
        "name": "Object Displacement Phenomenon",
        "description": "Demonstrates ability for limited physical interaction, typically harmless.",
    },
    {
        "level": 4,
        "name": "Thermokinetic Fluctuation",
        "description": "Presence is associated with noticeable temperature drops and psychosomatic discomfort.",
    },
    {
        "level": 5,
        "name": "Visual Entity",
        "description": "Capable of producing visual hallucinations without direct physical interaction.",
    },
    {
        "level": 6,
        "name": "Kinetic Manifestation",
        "description": "Shows ability to interact significantly with its environment, causing observable disruptions.",
    },
    {
        "level": 7,
        "name": "Aggressive Entity",
        "description": "Exhibits hostile behavior patterns, induces psychological distress.",
    },
    {
        "level": 8,
        "name": "Tactile Interaction",
        "description": "Demonstrates capability for direct physical contact with humans, causing increased fear response.",
    },
    {
        "level": 9,
        "name": "Harm-Inducing Specter",
        "description": "Exhibits behavior causing physical harm to humans, heightening risk factors.",
    },
    {
        "level": 10,
        "name": "Poltergeist Phenomenon",
        "description": "Exhibits extreme physical interactions and potential for high-level harm or damage.",
    },
]

### The ghost model and generating prompt

In [11]:
class Ghost(BaseModel):
    name_in_afterlife: str
    description_in_afterlife: str
    name_when_living: str
    description_when_living: str
    type_of_ghost: str
    scariness: int
    sex: str
    year_of_birth: int
    year_of_demise: int
    description_of_demise: str
    conditions_for_moving_on: str
    preferred_victims_to_scare: str
    preferred_method_to_scare: str


class GhostList(BaseModel):
    ghosts: list[Ghost]


pydantic_parser = PydanticOutputParser(pydantic_object=GhostList)

PROMPT_FORMAT = """You are a creative AI agent with a vivid imagination and a love of the macabre.
Create a list of ghosts who all reside within a haunted house with the following particulars.
```{house_info}```
Most ghosts will not be very scary, but some will be extremely scary.
The scariness of each ghost will conform to the following scale.
```{scariness_scale}```
{format_instructions}
The output should not deviate from this format in any way and every ghost should be listed.
No comments or ellipsis should appear in the output.
"""

prompt = PromptTemplate(
    template=PROMPT_FORMAT,
    input_variables=["house_info"],
    partial_variables={
        "scariness_scale": str(SCARINESS_SCALE),
        "format_instructions": pydantic_parser.get_format_instructions(),
    },
)
chain = LLMChain(llm=llm, prompt=prompt, output_parser=pydantic_parser, verbose=True)

In [12]:
ghost_list = chain.run(house_info=haunted_houses.haunted_houses[0].model_dump_json())



> Entering new LLMChain chain...
Prompt after formatting:
You are a creative AI agent with a vivid imagination and a love of the macabre.
Create a list of ghosts who all reside within a haunted house with the following particulars.
```{"name":"The Shadowy Estate","address":"123 Elm Street","type":"Victorian Mansion","description":"A hauntingly beautiful Victorian mansion with a sinister past.","number_of_ghosts":3,"year_built":1875,"year_abandoned":1940}```
Most ghosts will not be very scary, but some will be extremely scary.
The scariness of each ghost will conform to the following scale.
```[{'level': 1, 'name': 'Benevolent Specter', 'description': 'Exhibits non-threatening, socially positive behavior patterns.'}, {'level': 2, 'name': 'Auditory Manifestation', 'description': 'Engages primarily in acoustic disturbances, evoking mild unease.'}, {'level': 3, 'name': 'Object Displacement Phenomenon', 'description': 'Demonstrates ability for limited physical interaction, typically harml

In [13]:
ghosts_df = pd.DataFrame(map(BaseModel.model_dump, ghost_list.ghosts))
display(JSON(json.loads(haunted_houses.haunted_houses[0].model_dump_json())), ghosts_df)

<IPython.core.display.JSON object>

,name_in_afterlife,description_in_afterlife,name_when_living,description_when_living,type_of_ghost,scariness,sex,year_of_birth,year_of_demise,description_of_demise,conditions_for_moving_on,preferred_victims_to_scare,preferred_method_to_scare
0,The Gentle Lady,A graceful apparition often seen wandering the...,Eleanor Thomson,A kind-hearted woman known for her love of mus...,Benevolent Specter,1,Female,1840,1902,Eleanor passed away peacefully in her sleep.,Hearing the melody of her favorite song played...,"None, she is a benevolent spirit.",N/A
1,The Whispering Child,"A small spectral figure, often heard whisperin...",Timothy Smith,A mischievous child with a love for playing pr...,Auditory Manifestation,2,Male,1865,1872,Timothy tragically drowned in a nearby river.,Finding his lost teddy bear.,Those who are alone and in quiet places.,Whispering and giggling in the dark.
2,The Angry Butler,"A stern apparition, often seen moving objects ...",Gerald Hawkins,"A strict, disciplined man who served as the ma...",Object Displacement Phenomenon,3,Male,1820,1890,Gerald was killed in a kitchen accident involv...,"A proper burial for his remains, which are rum...",Those who disrupt the order of the house.,Moving objects to create confusion and disarray.


In [15]:
houses_ghost_lists = [
    (haunted_house, chain.run(house_info=haunted_house.model_dump_json()))
    for haunted_house in tqdm(haunted_houses.haunted_houses)
]

  0%|          | 0/29 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:
You are a creative AI agent with a vivid imagination and a love of the macabre.
Create a list of ghosts who all reside within a haunted house with the following particulars.
```{"name":"The Shadowy Estate","address":"123 Elm Street","type":"Victorian Mansion","description":"A hauntingly beautiful Victorian mansion with a sinister past.","number_of_ghosts":3,"year_built":1875,"year_abandoned":1940}```
Most ghosts will not be very scary, but some will be extremely scary.
The scariness of each ghost will conform to the following scale.
```[{'level': 1, 'name': 'Benevolent Specter', 'description': 'Exhibits non-threatening, socially positive behavior patterns.'}, {'level': 2, 'name': 'Auditory Manifestation', 'description': 'Engages primarily in acoustic disturbances, evoking mild unease.'}, {'level': 3, 'name': 'Object Displacement Phenomenon', 'description': 'Demonstrates ability for limited physical interaction, typically harml

## Merging the Haunted Houses and Ghosts Models

In [16]:
class PopulatedHauntedHouse(BaseModel):
    name: str
    address: str
    type: str
    description: str
    ghosts: list[Ghost]
    year_built: int
    year_abandoned: int


class PopulatedHauntedHouseListings(BaseModel):
    location: str
    haunted_houses: list[PopulatedHauntedHouse]

In [17]:
populated_haunted_house_listings = PopulatedHauntedHouseListings(
    location=haunted_houses.location,
    haunted_houses=[
        PopulatedHauntedHouse(
            **haunted_house.model_dump() | dict(ghosts=ghost_list.ghosts)
        )
        for haunted_house, ghost_list in houses_ghost_lists
    ],
)

In [18]:
with open("haunted_houses.json", "w", encoding="utf-8") as f:
    json.dump(
        populated_haunted_house_listings.model_dump_json(),
        f,
        ensure_ascii=False,
        indent=4,
    )

In [36]:
# with open("haunted_houses.json", "r") as f:
#     loaded_house_listings = json.load(f)
# loaded_house_listings = json.loads(loaded_house_listings)
# loaded_house_listings

### The Ghosts DataFrame

In [28]:
ghosts_df = pd.DataFrame(
    h.model_dump(exclude="ghosts") | g.model_dump()
    for h in populated_haunted_house_listings.haunted_houses
    for g in h.ghosts
)
ghosts_df

,name,address,type,description,year_built,year_abandoned,name_in_afterlife,description_in_afterlife,name_when_living,description_when_living,type_of_ghost,scariness,sex,year_of_birth,year_of_demise,description_of_demise,conditions_for_moving_on,preferred_victims_to_scare,preferred_method_to_scare
0,The Shadowy Estate,123 Elm Street,Victorian Mansion,A hauntingly beautiful Victorian mansion with ...,1875,1940,Lily the Lamenting Lady,"An ethereal lady in Victorian attire, often se...",Lily Thornton,A kind-hearted socialite famed for her beauty ...,Benevolent Specter,1,Female,1855,1877,Died of a broken heart after her lover mysteri...,To find closure about her lover's disappearance.,"She does not prefer to scare, but soothes anyo...","Appearing in mirrors in a sad, ghostly form."
1,The Shadowy Estate,123 Elm Street,Victorian Mansion,A hauntingly beautiful Victorian mansion with ...,1875,1940,Terrifying Thomas,"A tall, spectral figure in a top hat, known fo...",Thomas Blackwood,"A wealthy, stern businessman with a twisted se...",Auditory Manifestation,2,Male,1820,1890,"Died in his sleep, grinning after a successful...",To pull off one last grand prank.,Anyone who dares to live in his beloved mansion.,Laughing eerily and moving objects unexpectedly.
2,The Shadowy Estate,123 Elm Street,Victorian Mansion,A hauntingly beautiful Victorian mansion with ...,1875,1940,The Fearful Footman,"A ghostly servant seen wandering the halls, of...",Edward Harris,"A loyal footman with a timid nature, always ea...",Thermokinetic Fluctuation,4,Male,1840,1912,Died due to hypothermia while waiting outside ...,Recognition of his loyalty and service.,People who are disrespectful to the mansion.,Causing sudden cold gusts of wind and rattling...
3,The Whispering Farmhouse,456 Main Road,Farmhouse,An eerie farmhouse where whispers echo through...,1890,1960,The Whispering Widow,A spectral figure of a woman in period clothin...,Margaret Johnson,A loving wife and doting mother who spent her ...,Auditory Manifestation,2,female,1860,1897,"Margaret died of a mysterious illness, her whi...","Her husband's wedding ring, lost for years, ne...",Men who remind her of her husband.,Whispering cryptic warnings of impending doom.
4,The Whispering Farmhouse,456 Main Road,Farmhouse,An eerie farmhouse where whispers echo through...,1890,1960,The Cold Farmer,"An imposing, frosty specter of a man, often se...",Edward Johnson,"A hardworking farmer, husband to Margaret, who...",Thermokinetic Fluctuation,4,male,1852,1910,"Edward died of a broken heart, years after his...",He needs to find his wife's whispering spirit ...,Anyone who disrespects the farm or his wife's ...,Dramatic drops in temperature and the appearan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,The Gruesome Grange,951 Daisy Drive,Grange,A grange that sends a gruesome shiver down you...,1880,1940,Sobbing Sally,"Often found weeping in the attic, her sorrowfu...",Sally Thompson,A lonely spinster who lived in the house as a ...,Auditory Manifestation,2,Female,1854,1902,"Died alone in her attic room, heartbroken from...",Finding true love in the afterlife.,Unhappily married couples.,Sobbing loudly in the middle of the night.
79,The Gruesome Grange,951 Daisy Drive,Grange,A grange that sends a gruesome shiver down you...,1880,1940,Ghastly Gerald,"A ghost often seen flickering in the mirrors, ...",Gerald Richardson,"A stern and cruel man, who used to be the owne...",Visual Entity,5,Male,1820,1885,"Was found dead in his study, cause of death un...",Being forgiven by those he wronged in his life.,Young men who remind him of himself.,Appearing in mirrors when least expected.
80,The Gruesome Grange,951 Daisy Drive,Grange,A grange that sends a gruesome shiver down you...,1880,1940,Poltergeist Peter,"Infamous for causing havoc around the house, t...",Peter McMillan,A mischievous boy who lived in the house and l...,Poltergeist Phenomenon,10,Male,1930,1940,Died tragically in a freak accident

In [27]:
ghosts_df.groupby("scariness").name_in_afterlife.count()

scariness
1      6
2     24
3      5
4     18
5     10
6      7
7      4
9      1
10     8
Name: name_in_afterlife, dtype: int64

In [34]:
ghosts_df.groupby(["name", "address"]).name_in_afterlife.count()

name                         address             
The Apparitional Abode       357 Tulip Terrace       2
The Blood-curdling Bungalow  159 Daisy Drive         1
The Chilling Chalet          753 Orchid Oval         3
The Creepy Cabin             159 Hickory Heights     3
The Cursed Castle            951 Lilac Lane          3
The Dreaded Dwelling         951 Carnation Court     2
The Dreadful Bungalow        654 Willow Way          4
The Eerie Estate             159 Jasmine Junction    3
The Fearsome Farmhouse       753 Daisy Drive         2
The Frightening Farmstead    654 Magnolia Mews       3
The Ghastly Grange           864 Cedar Court         5
The Ghoulish Guesthouse      357 Holly Hill          2
The Gruesome Grange          951 Daisy Drive         3
The Haunting Hacienda        246 Birch Boulevard     2
The Morbid Mansion           258 Chestnut Chase      4
The Ominous Old House        951 Ivy Isle            4
The Petrifying Palace        357 Lily Lane           5
The Petrifying 

In [ ]:
# import asyncio


# async def async_generate(achain: LLMChain, haunted_house: HauntedHouse):
#     response = await achain.arun(house_info=haunted_house.model_dump_json())
#     return haunted_house, response


# async def generate_concurrently():
#     _llm = ChatOpenAI(model_name="gpt-4")
#     _chain = LLMChain(llm=_llm, prompt=prompt, output_parser=pydantic_parser)
#     tasks = [
#         await async_generate(achain=_chain, haunted_house=haunted_house)
#         for haunted_house in tqdm(haunted_houses.haunted_houses[:2])
#     ]
#     return await asyncio.gather(*tasks)


# houses_and_ghosts_lists = await generate_concurrently()